# 5 days of Data cleaning challenge.
## Day 1 : Handling missing values

Here is what we are going to do today :
* [Take a first look at your data](#1)
* [See how many data points we have](#2)
* [Figure out why the data is missing](#3)
* [Drop missing values](#4)
* [Filling in missing values](#5)

Let's gets started!

## For data click [here](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016)

## Take a first look at your data<a id="1"></a>
The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('/kaggle/input/nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv')

In [ ]:
df.head()

In [ ]:
# used to generate same sample everytime we run the code
np.random.seed(0) 

# used to generate a sample random row or column
df.sample(5) 

## See how many missing data points we have<a id="2"></a>
Ok, now we know that we do have some missing values. Let's see how many we have in each column.

In [ ]:
# get the number of missing values per column
missing = df.isnull().sum()

# look at the number of missing values in first 10 rows
missing[0:10]

That seems a lot!!! It might helpful to see what percentage of the values in our data set wer missing to give use a better sense of the scale of the problem.

In [ ]:
# How many total missing values we have?
total_cells = np.product(df.shape)
total_missing = missing.sum()

# percent of data that is missing
(total_missing / total_cells) * 100

Wow, almost a quater of the cells in this dataset is empty! Now we are going to take a closer look at some columns with missing values and try to figure out what might be going on with them.

## Figure out why data is missing<a id="3"></a>
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this : 

**Is this value missing because it wasn't recorded or because it dosen't exist?**

If a value is missing because it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probably do want to keep as NaN. On the other hand, if a value is missing because it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the df dataframe, I notice that the column `TimesSec` has a lot of missing values in it:

In [ ]:
# look at the # of missing points in the first ten columns
missing[0:10]

By looking at the documentation, I can see that `TimeSec` column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say which team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

**Tip** : This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

In [ ]:
df['PenalizedTeam'].isnull().value_counts()

## Drop missing values<a id="4"></a>
If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [ ]:
# removes all the rows that have missing values
df.dropna()

Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the columns that have at least one missing value instead.

In [ ]:
# remove all columns with atleast one missing value
columns_with_na_dropped = df.dropna(axis=1)
columns_with_na_dropped

In [ ]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % df.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

We've lost quite a bit of data, but at this point we have successfully removed all the NaN's from our data.

## Filling in missing values automatically<a id="5"></a>

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [ ]:
# get a small subset of NFL dataset
subset_df = df.loc[:, 'EPA' : 'Season'].head()
subset_df

We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the NaN values to be replaced with. Here, I'm saying that I would like to replace all the NaN values with 0.

In [ ]:
subset_df.fillna(0)

I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [ ]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_df.fillna(method = 'bfill', axis=0).fillna(0)